# 07. 커스텀 데이터셋(Custom Dataset)
앞 내용을 잠깐 복습해봅시다. 파이토치에서는 데이터셋을 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 torch.utils.data.Dataset과 torch.utils.data.DataLoader를 제공합니다. 이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있습니다. 기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것입니다.

## 1. 커스텀 데이터셋(Custom Dataset)
그런데 torch.utils.data.Dataset을 상속받아 직접 커스텀 데이터셋(Custom Dataset)을 만드는 경우도 있습니다. __torch.utils.data.Dataset은 파이토치에서 데이터셋을 제공하는 `추상 클래스`입니다.__ Dataset을 상속받아 다음 메소드들을 오버라이드 하여 커스텀 데이터셋을 만들어보겠습니다.

커스텀 데이터셋을 만들 때, 일단 가장 기본적인 뼈대는 아래와 같습니다. 여기서 필요한 기본적인 define은 3개입니다.

In [ ]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self):
        # 데이터셋의 전처리를 해주는 부분

    def __len__(self):
        # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분

    def __getitem__(self, idx): 
        # 데이터셋에서 특정 1개의 샘플을 가져오는 함수

- len(dataset)을 했을 때 데이터셋의 크기를 리턴할 len
- dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 get_item


***

## 2. 커스텀 데이터셋(Custom Dataset)으로 선형 회귀 구현하기


In [17]:
import torch
import torch.nn.functional as F

In [18]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [23]:
# Dataset 상속
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [35]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [25]:
model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [26]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
#         print(samples)
        x_train, y_train = samples
        prediction = model(x_train)
        
        cost  = F.mse_loss(prediction, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Batch{}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
        ))

Epoch    0/20 Batch1/3 Cost: 73945.468750
Epoch    0/20 Batch2/3 Cost: 12014.589844
Epoch    0/20 Batch3/3 Cost: 3161.554199
Epoch    1/20 Batch1/3 Cost: 1614.686523
Epoch    1/20 Batch2/3 Cost: 1117.088867
Epoch    1/20 Batch3/3 Cost: 307.931915
Epoch    2/20 Batch1/3 Cost: 43.864662
Epoch    2/20 Batch2/3 Cost: 17.931057
Epoch    2/20 Batch3/3 Cost: 13.595631
Epoch    3/20 Batch1/3 Cost: 3.521198
Epoch    3/20 Batch2/3 Cost: 1.447977
Epoch    3/20 Batch3/3 Cost: 0.678004
Epoch    4/20 Batch1/3 Cost: 1.538845
Epoch    4/20 Batch2/3 Cost: 2.502191
Epoch    4/20 Batch3/3 Cost: 2.008429
Epoch    5/20 Batch1/3 Cost: 3.347962
Epoch    5/20 Batch2/3 Cost: 1.355258
Epoch    5/20 Batch3/3 Cost: 0.742307
Epoch    6/20 Batch1/3 Cost: 0.436006
Epoch    6/20 Batch2/3 Cost: 2.072350
Epoch    6/20 Batch3/3 Cost: 4.963222
Epoch    7/20 Batch1/3 Cost: 1.282075
Epoch    7/20 Batch2/3 Cost: 2.952949
Epoch    7/20 Batch3/3 Cost: 0.812298
Epoch    8/20 Batch1/3 Cost: 0.464656
Epoch    8/20 Batch2/3 Cost:

In [31]:
print(len(dataset))
print(dataset[1])
for i in dataset:
    print(i)

5
(tensor([93., 88., 93.]), tensor([185.]))
(tensor([73., 80., 75.]), tensor([152.]))
(tensor([93., 88., 93.]), tensor([185.]))
(tensor([89., 91., 90.]), tensor([180.]))
(tensor([ 96.,  98., 100.]), tensor([196.]))
(tensor([73., 66., 70.]), tensor([142.]))


In [32]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 

# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 

print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[153.0047]], grad_fn=<AddmmBackward>)
